In [ ]:
import cv2
import numpy as np
import mediapipe as mp

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

Angle Function

In [ ]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - \
              np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle
    return angle

Detect Thumbs Up Gesture

In [ ]:
def is_thumb_up(lm):
    thumb_up = lm.landmark[4].y < lm.landmark[3].y

    # fingers folded check
    folded = all(
        lm.landmark[tip].y > lm.landmark[tip-2].y  # tip > pip → folded
        for tip in [8, 12, 16, 20]
    )

    return thumb_up and folded


Hand Menu (Thumbs Up Detection)

In [ ]:
def gesture_menu(cap):
    print("\nShow LEFT thumbs up for Left Arm curls")
    print("Show RIGHT thumbs up for Right Arm curls\n")

    with mp_hands.Hands(max_num_hands=2,
                        min_detection_confidence=0.3,
                        min_tracking_confidence=0.8) as hands:

        while True:
            ret, frame = cap.read()
            if not ret:
                continue

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(image)

            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.multi_hand_landmarks:
                for hand_landmarks, handedness in zip(
                        results.multi_hand_landmarks,
                        results.multi_handedness):

                    mp_drawing.draw_landmarks(
                        image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    label = handedness.classification[0].label  # Left/Right

                    if is_thumb_up(hand_landmarks):
                        print(f">>> {label.upper()} THUMBS UP DETECTED")

                        if label == "Left":
                            return "RIGHT"
                        else:
                            return "LEFT"

            cv2.putText(image, "Show LEFT or RIGHT thumbs-up",
                        (40, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                        (0, 255, 0), 2)

            cv2.imshow("Gesture Menu", image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                
                return None
        cap.release()
        cv2.destroyAllWindows()

 MAIN WORKOUT

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    arm_selected = gesture_menu(cap)
    print("\nSelected:", arm_selected)

    counter = 0
    stage = None
    set_num = 1

    with mp_pose.Pose(min_detection_confidence=0.3,
                       min_tracking_confidence=0.8) as pose:

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark

                if arm_selected == "LEFT":
                    shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
                    wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

                else:
                    shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
                    wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]

                shoulder = [shoulder.x, shoulder.y]
                elbow = [elbow.x, elbow.y]
                wrist = [wrist.x, wrist.y]

                angle = calculate_angle(shoulder, elbow, wrist)

                cv2.putText(image, str(int(angle)),
                            tuple((np.multiply(elbow, [640, 480])).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)

                if angle > 160:
                    stage = "down"
                if angle < 40 and stage == "down":
                    stage = "up"
                    counter += 1
                    print("Reps:", counter)

                # Set logic
                if 0 < counter <= 10: set_num = 1
                elif 10 < counter <= 20: set_num = 2
                elif 20 < counter <= 30: set_num = 3

                if counter >= 30:
                    print("\n3 sets completed. Returning to gesture menu...")
                    break

            cv2.rectangle(image, (0, 0), (250, 80),
                          (245,117,16), -1)

            cv2.putText(image, f"ARM: {arm_selected}",
                        (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                        (255,255,255), 2)
            cv2.putText(image, f"SET: {set_num}",
                        (10, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                        (255,255,255), 2)
            cv2.putText(image, f"REPS: {counter}",
                        (130, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                        (255,255,255), 2)

            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            cv2.imshow("Workout", image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                
                break

        cap.release()
        cv2.destroyAllWindows()

            
